<a href="https://colab.research.google.com/github/Manditar2/NLI-con-BiLSTM/blob/main/Probandi_BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import keras.backend as K
import nltk
from tensorflow import keras
from keras.models import Model
from keras.optimizers import RMSprop
from keras.regularizers import l2
from keras.layers import Input, Embedding, Dense, Dropout, TimeDistributed, BatchNormalization, Bidirectional, LSTM, concatenate
from keras.layers import BatchNormalization, Lambda, concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
import tempfile

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

In [ ]:
nltk.download('stopwords')
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


--2023-06-19 16:35:43--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-06-19 16:35:44--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.04MB/s    in 2m 40s  

2023-06-19 16:38:24 (5.14 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflatin

In [ ]:
#@title Lectura de datos
df_entrenamiento = pd.read_csv('/content/sample_data/snli_1.0_train.csv')
df_pruebas = pd.read_csv('/content/sample_data/snli_1.0_test.csv')
df_validacion = pd.read_csv('/content/sample_data/snli_1.0_dev.csv')

In [ ]:
columnas = ['sentence1','sentence2','gold_label']
entrenamiento = df_entrenamiento[columnas]
pruebas = df_pruebas[columnas]
validacion = df_validacion[columnas]

entrenamiento.dropna(axis = 0, inplace = True)
pruebas.dropna(axis = 0, inplace = True)
validacion.dropna(axis = 0, inplace = True)

<ipython-input-6-ab25479910e7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entrenamiento.dropna(axis = 0, inplace = True)
<ipython-input-6-ab25479910e7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pruebas.dropna(axis = 0, inplace = True)
<ipython-input-6-ab25479910e7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validacion.dropna(axis = 0, inplace = True)


In [ ]:

entrenamiento = entrenamiento.loc[entrenamiento['gold_label'] != "-"]
entrenamiento = entrenamiento.loc[entrenamiento['gold_label'] != "neutral"]

pruebas = pruebas.loc[pruebas['gold_label'] != "-"]
pruebas = pruebas.loc[pruebas['gold_label']!= "neutral"]
pruebas


,sentence1,sentence2,gold_label
1,This church choir sings to the masses as they ...,The church is filled with song.,entailment
2,This church choir sings to the masses as they ...,A choir singing at a baseball game.,contradiction
4,"A woman with a green headscarf, blue shirt and...",The woman is very happy.,entailment
5,"A woman with a green headscarf, blue shirt and...",The woman has been shot.,contradiction
6,An old man with a package poses in front of an...,A man poses in front of an ad.,entailment
...,...,...,...
9992,A female dancer wearing all white dancing whil...,Some players are playing cricket in ground,contradiction
9995,Two women are observing something together.,Two women are standing with their eyes closed.,contradiction
9996,Two women are observing something together.,Two girls are looking at something.,entailment
9997,A man in a black leather jacket and a book in ...,A man is flying a kite.,contradiction


In [ ]:
mapeo_label = {
    "neutral":0,
    "contradiction":1,
    "entailment":2
}
entrenamiento['gold_label'] = entrenamiento['gold_label'].map(mapeo_label)
pruebas['gold_label'] = pruebas['gold_label'].map(mapeo_label)

In [ ]:
#@title Limpieza de datos
def limpiarTexto(texto):
  texto = str(texto).lower()
  texto = re.sub('\[.*?\]', '', texto)
  texto = re.sub('<.*?>+', '', texto)
  texto = re.sub('\n', '', texto)
  texto = re.sub('[%s]' % re.escape(string.punctuation), '', texto)
  texto = re.sub('\w*\d\w*', '', texto)
  return texto

entrenamiento['sentence1'] = entrenamiento['sentence1'].apply(limpiarTexto)
entrenamiento['sentence2'] = entrenamiento['sentence2'].apply(limpiarTexto)

pruebas['sentence2'] = pruebas['sentence2'].apply(limpiarTexto)
pruebas['sentence1'] = pruebas['sentence1'].apply(limpiarTexto)

pruebas

,sentence1,sentence2,gold_label
1,this church choir sings to the masses as they ...,the church is filled with song,2
2,this church choir sings to the masses as they ...,a choir singing at a baseball game,1
4,a woman with a green headscarf blue shirt and ...,the woman is very happy,2
5,a woman with a green headscarf blue shirt and ...,the woman has been shot,1
6,an old man with a package poses in front of an...,a man poses in front of an ad,2
...,...,...,...
9992,a female dancer wearing all white dancing whil...,some players are playing cricket in ground,1
9995,two women are observing something together,two women are standing with their eyes closed,1
9996,two women are observing something together,two girls are looking at something,2
9997,a man in a black leather jacket and a book in ...,a man is flying a kite,1


In [ ]:
stop_words = stopwords.words('english')
more_stopwords = ['u', 'im', 'c']
stop_words = stop_words + more_stopwords

def remove_stopwords(text):
    text = ' '.join(word for word in text.split(' ') if word not in stop_words)
    return text
entrenamiento['sentence1'] = entrenamiento['sentence1'].apply(remove_stopwords)
entrenamiento['sentence2'] = entrenamiento['sentence2'].apply(remove_stopwords)

pruebas['sentence1'] = pruebas['sentence1'].apply(remove_stopwords)
pruebas['sentence2'] = pruebas['sentence2'].apply(remove_stopwords)

pruebas

,sentence1,sentence2,gold_label
1,church choir sings masses sing joyous songs bo...,church filled song,2
2,church choir sings masses sing joyous songs bo...,choir singing baseball game,1
4,woman green headscarf blue shirt big grin,woman happy,2
5,woman green headscarf blue shirt big grin,woman shot,1
6,old man package poses front advertisement,man poses front ad,2
...,...,...,...
9992,female dancer wearing white dancing photograph...,players playing cricket ground,1
9995,two women observing something together,two women standing eyes closed,1
9996,two women observing something together,two girls looking something,2
9997,man black leather jacket book hand speaks clas...,man flying kite,1


In [ ]:
#stemmer VEREMOS SI SE APLICA O NO

In [ ]:
#@title Tokenización
tokens_entrenamiento = np.asarray([entrenamiento['sentence1'],entrenamiento['sentence2']])
tokens_entrenamiento = tokens_entrenamiento.reshape(-1,1)
tokens_prueba = np.asarray([pruebas['sentence1'],pruebas['sentence2']])
tokens_entrenamiento = tokens_entrenamiento.reshape(tokens_entrenamiento.shape[0])

In [ ]:
tokenizador = Tokenizer(num_words = 6000)
tokenizador.fit_on_texts(tokens_entrenamiento)
tokenizador

In [ ]:
premisa = tokenizador.texts_to_sequences(entrenamiento['sentence1'])
premisa = pad_sequences(premisa)

hipotesis = tokenizador.texts_to_sequences(entrenamiento['sentence2'])
hipotesis = pad_sequences(hipotesis, maxlen=premisa.shape[1])


In [ ]:
premisa_prueba = tokenizador.texts_to_sequences(pruebas['sentence1'])
premisa_prueba = pad_sequences(premisa_prueba,45)

hipotesis_prueba = tokenizador.texts_to_sequences(pruebas['sentence2'])
hipotesis_prueba = pad_sequences(hipotesis_prueba,maxlen = premisa_prueba.shape[1])

etiquetas_prueba = to_categorical(pruebas['gold_label'])

In [ ]:
#@title Etiquetas
etiquetas = to_categorical(entrenamiento['gold_label'])
etiquetas

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)

In [ ]:
#@title Embedding
vocabulario = len(tokenizador.word_index)

def gloveWork(archivo,vocabulario):
  #Se crea un diccionario palabra-vector de un modelo preentrenado(Glove)
  embeddings_index = {}
  file_ = open(archivo)
  for line in file_:
      arr = line.split()
      palabra = arr[0]
      vector = np.asarray(arr[1:],dtype='float32')
      embeddings_index[palabra] = vector
  file_.close()

  tam_maximo = vocabulario +1
  indices_palabras = tokenizador.word_index
  embedding_matriz = np.zeros((tam_maximo,300)).astype(object)

  for palabrax, indice in indices_palabras.items():
    vector_entrenado = embeddings_index.get(palabrax)
    if vector_entrenado is not None:
      embedding_matriz[indice] = vector_entrenado

  return embedding_matriz


In [ ]:
matriz_embedding = gloveWork('glove.6B.300d.txt',vocabulario)

In [ ]:
matriz_embedding.shape

(11161, 300)

In [ ]:
#@title Hiperparámetros
dropout = 0.2
L2 = 4e-6
learning_rate = 0.01
paciencia = 4
rho = 0.9
epsilon = 1e-08
decay = 0.0
validacion_split = 0.02
epoch = 20

In [ ]:
embedding = Embedding(input_dim=matriz_embedding.shape[0],
                          output_dim=300,
                          weights=[matriz_embedding],
                          input_length=42,
                          trainable=False)
BiLSTM = Bidirectional(LSTM(64))
translation = TimeDistributed(Dense(300, activation='relu'))
premise = Input(shape=(45,), dtype='int32')
hypothesis = Input(shape=(45,), dtype='int32')
premise_embedded = embedding(premise)
hypothesis_embedded = embedding(hypothesis)
premise_translated = translation(premise_embedded)
hypothesis_translated = translation(hypothesis_embedded)
premise_BiLSTM = BiLSTM(premise_translated)
hypothesis_BiLSTM = BiLSTM(hypothesis_translated)
premise_normalized = BatchNormalization()(premise_BiLSTM)
hypothesis_normalized = BatchNormalization()(hypothesis_BiLSTM)

train_input = concatenate([premise_normalized, hypothesis_normalized])
train_input = Dropout(dropout)(train_input)
# Apply the (Dense layer, Dropout layer. Batch normalization layer) unit : 1
train_input = Dense(2 * 300, activation='relu', kernel_regularizer=l2(L2))(train_input)
train_input = Dropout(dropout)(train_input)
train_input = BatchNormalization()(train_input)

# Apply the (Dense layer, Dropout layer. Batch normalization layer) unit : 2
train_input = Dense(2 * 300, activation='relu', kernel_regularizer=l2(L2))(train_input)
train_input = Dropout(dropout)(train_input)
train_input = BatchNormalization()(train_input)

# Apply the (Dense layer, Dropout layer. Batch normalization layer) unit : 3
train_input = Dense(2 * 300, activation='relu', kernel_regularizer=l2(L2))(train_input)
train_input = Dropout(dropout)(train_input)
train_input = BatchNormalization()(train_input)

prediction = Dense(3, activation='softmax')(train_input)
model = Model(inputs=[premise, hypothesis], outputs=prediction)
optimizer = RMSprop(lr=learning_rate, rho=rho, epsilon=epsilon, decay=decay)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 45)]         0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 45)]         0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 45, 300)      3348300     ['input_3[0][0]',                
                                                                  'input_4[0][0]']                
                                                                                                  
 time_distributed_1 (TimeDistri  (None, 45, 300)     90300       ['embedding_1[0][0]',      

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy',
                                            patience=paciencia,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss',patience=paciencia)
_, tmpfn = tempfile.mkstemp()
model_checkpoint = ModelCheckpoint(tmpfn, save_best_only=True, save_weights_only=True)
callbacks = [early_stopping, model_checkpoint, learning_rate_reduction]

history = model.fit(x=[premisa, hipotesis],
                    y=etiquetas,
                    batch_size=512,
                    epochs=epoch,
                    validation_split=validacion_split,
                    callbacks=callbacks)


Epoch 1/20
73/73 [==============================] - 169s 2s/step - loss: 1.0594 - accuracy: 0.5880 - val_loss: 0.6595 - val_accuracy: 0.6288 - lr: 0.0100
Epoch 2/20
73/73 [==============================] - 156s 2s/step - loss: 0.6332 - accuracy: 0.6978 - val_loss: 0.8935 - val_accuracy: 0.6552 - lr: 0.0100
Epoch 3/20
73/73 [==============================] - 156s 2s/step - loss: 0.5432 - accuracy: 0.7373 - val_loss: 0.6359 - val_accuracy: 0.6737 - lr: 0.0100
Epoch 4/20
73/73 [==============================] - 152s 2s/step - loss: 0.5093 - accuracy: 0.7573 - val_loss: 0.5696 - val_accuracy: 0.6830 - lr: 0.0100
Epoch 5/20
73/73 [==============================] - 154s 2s/step - loss: 0.4747 - accuracy: 0.7774 - val_loss: 0.5634 - val_accuracy: 0.7781 - lr: 0.0100
Epoch 6/20
73/73 [==============================] - 156s 2s/step - loss: 0.4506 - accuracy: 0.7907 - val_loss: 0.7471 - val_accuracy: 0.7001 - lr: 0.0100
Epoch 7/20
73/73 [==============================] - 154s 2s/step - loss: 0.4

In [ ]:
datos_prueba = [premisa_prueba, hipotesis_prueba]
evaluacion = model.evaluate(datos_prueba,etiquetas_prueba)

207/207 [==============================] - 19s 91ms/step - loss: 0.5310 - accuracy: 0.7785
